In [ ]:
%%capture

!pip install tensorflow-addons

In [ ]:
import pandas as pd
import os
from tqdm.notebook import tqdm
pd.set_option('max_columns', None)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import gc
gc.enable()
from sklearn import preprocessing, model_selection, metrics
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.metrics import confusion_matrix
import tensorflow_addons as tfa
from warnings import filterwarnings
filterwarnings('ignore')
from scipy.stats import mode

from sklearn.metrics import roc_curve, auc,roc_auc_score
import keras
from tqdm import tqdm
import tensorflow_addons as tfa

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

![](https://i.pinimg.com/736x/99/74/66/997466bf627d1ca635861b9ceb5b4ed7.jpg)

# Version

### **v1** -> Single Hidden Layer One Neuron Only Prediction (callback typo)
### **v2** -> Single Hidden Layer One Neuron Only Prediction
### **v3** -> Spider-web DNN Model XD

In [ ]:
#notebook config
class config:
    target = 'target'
    working_dir = '../input/tabular-playground-series-feb-2022/'
    seed = 42
    folds = 5
    plot = False
    target_map = {'Bacteroides_fragilis': 0,
                 'Campylobacter_jejuni': 1,
                 'Enterococcus_hirae': 2,
                 'Escherichia_coli': 3,
                 'Escherichia_fergusonii': 4,
                 'Klebsiella_pneumoniae': 5,
                 'Salmonella_enterica': 6,
                 'Staphylococcus_aureus': 7,
                 'Streptococcus_pneumoniae': 8,
                 'Streptococcus_pyogenes': 9}
    
    target_remap = {0: 'Bacteroides_fragilis',
                     1: 'Campylobacter_jejuni',
                     2: 'Enterococcus_hirae',
                     3: 'Escherichia_coli',
                     4: 'Escherichia_fergusonii',
                     5: 'Klebsiella_pneumoniae',
                     6: 'Salmonella_enterica',
                     7: 'Staphylococcus_aureus',
                     8: 'Streptococcus_pneumoniae',
                     9: 'Streptococcus_pyogenes'}
    name = 'DNN'
    device = 'GPU'
    classes = 10
    
    # define callbacks
    lr = keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", 
        factor=0.2, 
        patience=5, 
        verbose=True,
    )

    es = keras.callbacks.EarlyStopping(
        monitor="val_f1_score", 
        patience=30, 
        verbose=True, 
        mode="max",
        restore_best_weights=True
    )
    
    callback = [lr, es]
    
    metric = f1 = tfa.metrics.F1Score(config.classes, 'weighted')
    
np.random.seed(config.seed)
random.seed(config.seed)
tf.random.set_seed(config.seed)

In [ ]:
#reduce memory
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
#importing dataset
train = pd.read_csv(config.working_dir + 'train.csv', index_col='row_id')
test = pd.read_csv(config.working_dir + 'test.csv', index_col='row_id')

train = train.drop_duplicates().reset_index(drop=True)

y = train[config.target]
train = train.drop(columns = config.target)

y = pd.get_dummies(y.copy())
y.columns = list(config.target_remap.values())

sample_submission = pd.read_csv(config.working_dir + 'sample_submission.csv')

In [ ]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    tf_strategy = tf.distribute.experimental.TPUStrategy(tpu)
    print("Running on TPU:", tpu.master())
except:
    tf_strategy = tf.distribute.get_strategy()
    print(f"Running on {tf_strategy.num_replicas_in_sync} replicas")
    print("Number of GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
from keras.layers import Dense, Input, BatchNormalization, Add, Concatenate, Multiply, Dropout, Conv1D, Flatten, MaxPooling1D, Subtract
from tensorflow.keras.regularizers import L1, L2

def get_model():
    keras.backend.clear_session()
    
    #one neuron
    input_ = Input(shape = (train.shape[1],))
    x = Dense(664, activation = 'elu', kernel_initializer= 'he_normal')((input_))
    x = BatchNormalization()(x)
    
    x1 = Dense(216, activation = 'elu', kernel_initializer= 'he_normal')(x)
    xmerger = x1
    x1 = Dropout(0.45)(x1)
    x1 = BatchNormalization()(x1)
    
    x1 = Dense(164, activation = 'elu', kernel_initializer= 'he_normal')(x1)
    xkeeper = x1 #164
    x1 = Dropout(0.25)(x1)
    x1 = BatchNormalization()(x1)
    
    x1 = Dense(164, activation = 'elu', kernel_initializer= 'he_normal')(x1)
    x1 = Dropout(0.25)(x1)
    x1 = BatchNormalization()(x1)
    
    x1 = Dense(164, activation = 'elu', kernel_initializer= 'he_normal')(x1)
    x1 = Dropout(0.25)(x1)
    x1 = BatchNormalization()(x1)
    
    xmul = Multiply()([x1, xkeeper])
    xadd = Add()([x1, xkeeper])
    xsub = Subtract()([x1, xkeeper])
    
    xx = Concatenate()([xkeeper, xmul, xadd, xsub, x1]) 
    
    x1 = Dense(624, activation = 'elu', kernel_initializer= 'he_normal')(xx)
    x1 = Dropout(0.45)(x1)
    x1 = BatchNormalization()(x1)
    
    x1 = Dense(216, activation = 'elu', kernel_initializer= 'he_normal')(x1)
    x1 = Add()([x1,xmerger])
    x1 = Dropout(0.35)(x1)
    x1 = BatchNormalization()(x1)
    
    x = Dense(config.classes, activation = 'softmax')(x1)
    
    #model
    model = keras.Model(inputs = [input_], outputs = [x])
    
    return model

In [ ]:
!pip install visualkeras
# import visualkeras

In [ ]:
import visualkeras
visualkeras.layered_view(get_model())

In [ ]:
tf.keras.utils.plot_model(get_model(), show_dtype= True, show_layer_names=True)

In [ ]:
%%time
# create dictionaries to store predictions
test_pred_tmp = dict()
scores_tmp = dict()

name = config.name
config.metric = metrics.accuracy_score

test_pred_tmp[name] = list()
scores_tmp[name] = list()


config.scaler = RobustScaler()

# create cv
kf = StratifiedKFold(n_splits=config.folds, shuffle=True, random_state=1)

for fold, (idx_train, idx_valid) in enumerate(kf.split(train, y.idxmax(axis =1).map(config.target_map))):
    model = get_model()
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss= 'CategoricalCrossentropy',
        metrics=[config.f1])
    
    # create train, validation sets
    X_train, y_train = train.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = train.iloc[idx_valid], y.iloc[idx_valid]
    
    #preprocess
    scaler = config.scaler

    X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
    X_valid = pd.DataFrame(scaler.transform(X_valid), columns = X_valid.columns)
    test_scaled = pd.DataFrame(scaler.transform(test), columns = test.columns)
    
    
    history = model.fit(X_train,y_train,validation_data=(X_valid, y_valid),
        callbacks=config.callback, epochs=10000,batch_size=2024,
        shuffle=True, verbose=1
    )

    lol = pd.DataFrame(history.history['f1_score']).plot(label='f1_score')
    pd.DataFrame(history.history['val_f1_score']).plot(ax=lol, label='val_f1_score')
    
    plt.legend(['f1_score','val_f1_score'])

    plt.show()
    # validation prediction
    score = np.max(history.history['val_f1_score'])

    scores_tmp[name].append(score)

    print(f"Fold: {fold + 1} Model: {name} Score: {score}")
    print('--'*20)
    # test prediction
    y_hat = model.predict(test_scaled)
    test_pred_tmp[name].append(y_hat)

    gc.collect()
    # print overall validation scores
# for name, model in models:
print(f"Overall Validation Score | {name}: {np.mean(scores_tmp[name])}")
print('::'*20)

In [ ]:
def post_process(test_pred_tmp):
    temp = [np.argmax(pred, axis = 1) for pred in test_pred_tmp[name]]
    temp = mode(temp, axis =0)[0].T
    temp = pd.DataFrame(np.squeeze(temp), columns = [config.target])
    temp = temp[config.target].map(config.target_remap)
    
    return temp

In [ ]:
sub = sample_submission.copy()
sub[config.target] = post_process(test_pred_tmp)
sub.to_csv('submission.csv', index = False)